In [1]:
# Names of datasets to be used
# '/Users/darionguan/Desktop/Presidential Prediction/acs_state_final.csv'
# '/Users/darionguan/Desktop/Presidential Prediction/acs_cd_final.csv'
# '/Users/darionguan/Desktop/Presidential Prediction/'biden.csv'
# '/Users/darionguan/Desktop/Presidential Prediction/trump.csv'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import pandasql as psql

In [3]:
fips_dict = {'AL': '01',
              'AK': '02',
              'AZ': '04',
              'AR': '05',
              'CA': '06',
              'CO': '08',
              'CT': '09',
              'DE': '10',
              'DC': '11',
              'FL': '12',
              'GA': '13',
              'HI': '15',
              'ID': '16',
              'IL': '17',
              'IN': '18',
              'IA': '19',
              'KS': '20',
              'KY': '21',
              'LA': '22',
              'ME': '23',
              'MD': '24',
              'MA': '25',
              'MI': '26',
              'MN': '27',
              'MS': '28',
              'MO': '29',
              'MT': '30',
              'NE': '31',
              'NV': '32',
              'NH': '33',
              'NJ': '34',
              'NM': '35',
              'NY': '36',
              'NC': '37',
              'ND': '38',
              'OH': '39',
              'OK': '40',
              'OR': '41',
              'PA': '42',
              'RI': '44',
              'SC': '45',
              'SD': '46',
              'TN': '47',
              'TX': '48',
              'UT': '49',
              'VT': '50',
              'VA': '51',
              'WA': '53',
              'WV': '54',
              'WI': '55',
              'WY': '56'}
inv_map = {v: k for k, v in fips_dict.items()}
fips_dict = inv_map
print(fips_dict)

{'01': 'AL', '02': 'AK', '04': 'AZ', '05': 'AR', '06': 'CA', '08': 'CO', '09': 'CT', '10': 'DE', '11': 'DC', '12': 'FL', '13': 'GA', '15': 'HI', '16': 'ID', '17': 'IL', '18': 'IN', '19': 'IA', '20': 'KS', '21': 'KY', '22': 'LA', '23': 'ME', '24': 'MD', '25': 'MA', '26': 'MI', '27': 'MN', '28': 'MS', '29': 'MO', '30': 'MT', '31': 'NE', '32': 'NV', '33': 'NH', '34': 'NJ', '35': 'NM', '36': 'NY', '37': 'NC', '38': 'ND', '39': 'OH', '40': 'OK', '41': 'OR', '42': 'PA', '44': 'RI', '45': 'SC', '46': 'SD', '47': 'TN', '48': 'TX', '49': 'UT', '50': 'VT', '51': 'VA', '53': 'WA', '54': 'WV', '55': 'WI', '56': 'WY'}


In [4]:
# Set up file name
filename = '/Users/darionguan/Desktop/Presidential Prediction/ZIP_CD_122019.csv'
# Read .csv and set up ZIP column as five number format with leading zeros
zip_df = pd.read_csv(filename, converters={'ZIP': '{:0>5}'.format})

# Insert STATE_ID column as substring of CD
zip_df.insert(1, 'STATE_ID', zip_df['CD'].str[0:2])
# Substring CD column to last two characters
zip_df['CD'] = zip_df['CD'].str[-2:]

# Insert STATE (abbreviation) column
zip_df['STATE'] = zip_df['STATE_ID'].map(fips_dict)

# Drop the below columns
zip_df = zip_df[(zip_df['STATE_ID'] != '66')]
zip_df = zip_df[(zip_df['STATE_ID'] != '72')]
zip_df = zip_df[(zip_df['STATE_ID'] != '78')]

# Sort by ZIP and RES_RATIO
zip_df = zip_df.sort_values(['ZIP', 'RES_RATIO'])

# Drop duplicates and keep last (keep highest RES_RATIO)
zip_df = zip_df.drop_duplicates(['ZIP', 'STATE_ID'], keep='last')

# Subset by below columns
zip_df = zip_df[['ZIP', 'STATE', 'STATE_ID', 'CD']]

# Reset index
zip_df = zip_df.reset_index(drop=True)

zip_df

,ZIP,STATE,STATE_ID,CD
0,00501,NY,36,01
1,01001,MA,25,01
2,01002,MA,25,02
3,01003,MA,25,02
4,01004,MA,25,02
...,...,...,...,...
39275,99925,AK,02,00
39276,99926,AK,02,00
39277,99927,AK,02,00
39278,99928,AK,02,00


In [5]:
# Read biden_df csv
filename = '/Users/darionguan/Desktop/Presidential Prediction/biden.csv'
biden_df = pd.read_csv(filename)

/var/folders/k1/z3v9vvcx2zj8knzj63hcxqh40000gn/T/ipykernel_46708/1780420249.py:3: DtypeWarning: Columns (16,18) have mixed types. Specify dtype option on import or set low_memory=False.
  biden_df = pd.read_csv(filename)


In [6]:
# Subset to STATE, ZIP_CODE, and TRANSACTION_AMT columns
biden_df = biden_df[['STATE', 'ZIP_CODE', 'TRANSACTION_AMT']]

# Remove NaN's for ZIP_CODE and STATE columns
biden_df = biden_df[biden_df['ZIP_CODE'].notna()]
biden_df = biden_df[biden_df['STATE'].notna()]

# Remove these as they are not states
biden_df = biden_df[(biden_df['STATE'] != 'ZZ')]
biden_df = biden_df[(biden_df['STATE'] != 'AE')]
biden_df = biden_df[(biden_df['STATE'] != 'VI')]
biden_df = biden_df[(biden_df['STATE'] != 'GU')]
biden_df = biden_df[(biden_df['STATE'] != 'PR')]
biden_df = biden_df[(biden_df['STATE'] != 'AP')]
biden_df = biden_df[(biden_df['STATE'] != 'AA')]
biden_df = biden_df[(biden_df['STATE'] != 'AS')]
biden_df = biden_df[(biden_df['STATE'] != 'PW')]
biden_df = biden_df[(biden_df['STATE'] != 'MP')]
biden_df = biden_df[(biden_df['STATE'] != 'FM')]


# Change dtype for manipulation
biden_df['ZIP_CODE'] = biden_df['ZIP_CODE'].apply(np.int64)
biden_df['ZIP_CODE'] = biden_df['ZIP_CODE'].apply(str)

# If ZIP_CODE is greater than 5 characters, remove the last four characters
biden_df.loc[biden_df['ZIP_CODE'].str.len() > 5, 'ZIP_CODE'] = biden_df['ZIP_CODE'].str[:-4]
biden_df['ZIP_CODE'] = biden_df['ZIP_CODE'].apply('{:0>5}'.format)

# Sort values by ZIP_CODE and STATE for viewing
biden_df = biden_df.sort_values(['ZIP_CODE', 'STATE'])

# Change dtype for manipulation
biden_df['ZIP_CODE'] = biden_df['ZIP_CODE'].apply(np.int64)

# Remove invalid zip codes
biden_df = biden_df[(biden_df['ZIP_CODE'] > 764) & (biden_df['ZIP_CODE'] < 99950)]

# Reformat ZIP_CODE to five characters
biden_df['ZIP_CODE'] = biden_df['ZIP_CODE'].apply(lambda x: '{0:0>5}'.format(x))

# Reset index
biden_df = biden_df.reset_index(drop=True)

biden_df

,STATE,ZIP_CODE,TRANSACTION_AMT
0,MA,01001,10
1,MA,01001,20
2,MA,01001,14
3,MA,01001,50
4,MA,01001,50
...,...,...,...
5611161,AK,99929,25
5611162,AK,99929,25
5611163,AK,99929,50
5611164,AK,99929,15


In [7]:
q = """
SELECT b.ZIP_CODE, b.STATE, b.TRANSACTION_AMT, z.STATE_ID, z.CD
FROM biden_df as b
LEFT JOIN zip_df as z
    ON b.ZIP_CODE = z.ZIP AND b.STATE = z.STATE
"""
biden_df = psql.sqldf(q)

# Group TRANSACTION_AMT by below values, sort, reset index
biden_df = biden_df.groupby(['STATE', 'STATE_ID', 'CD'])['TRANSACTION_AMT'].mean().to_frame().reset_index()
biden_df = biden_df.sort_values(['STATE_ID', 'CD'])
biden_df = biden_df.reset_index(drop=True)

biden_df.loc[biden_df['CD'] == '00', 'CD'] = '01'
biden_df

,STATE,STATE_ID,CD,TRANSACTION_AMT
0,AL,01,01,153.441706
1,AL,01,02,110.557545
2,AL,01,03,68.553998
3,AL,01,04,116.225519
4,AL,01,05,100.808035
...,...,...,...,...
431,WI,55,05,124.336661
432,WI,55,06,132.124739
433,WI,55,07,80.208220
434,WI,55,08,108.448556


In [8]:
filename = '/Users/darionguan/Desktop/Presidential Prediction/trump.csv'
trump_df = pd.read_csv(filename)

/var/folders/k1/z3v9vvcx2zj8knzj63hcxqh40000gn/T/ipykernel_46708/1507606538.py:2: DtypeWarning: Columns (5,15,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  trump_df = pd.read_csv(filename)


In [9]:
# Subset to STATE, ZIP_CODE, and TRANSACTION_AMT columns
trump_df = trump_df[['STATE', 'ZIP_CODE', 'TRANSACTION_AMT']]

# Remove NaN's for ZIP_CODE and STATE columns
trump_df = trump_df[trump_df['ZIP_CODE'].notna()]
trump_df = trump_df[trump_df['STATE'].notna()]

# # Remove these as they are not states
trump_df = trump_df[(trump_df['STATE'] != 'AA')]
trump_df = trump_df[(trump_df['STATE'] != 'AE')]
trump_df = trump_df[(trump_df['STATE'] != 'AP')]
trump_df = trump_df[(trump_df['STATE'] != 'AS')]
trump_df = trump_df[(trump_df['STATE'] != 'GU')]
trump_df = trump_df[(trump_df['STATE'] != 'MP')]
trump_df = trump_df[(trump_df['STATE'] != 'PR')]
trump_df = trump_df[(trump_df['STATE'] != 'VI')]
trump_df = trump_df[(trump_df['STATE'] != 'ZZ')]


# Change dtype for manipulation
trump_df['ZIP_CODE'] = trump_df['ZIP_CODE'].apply(np.int64)
trump_df['ZIP_CODE'] = trump_df['ZIP_CODE'].apply(str)

# If ZIP_CODE is greater than 5 characters, remove the last four characters
trump_df.loc[trump_df['ZIP_CODE'].str.len() > 5, 'ZIP_CODE'] = trump_df['ZIP_CODE'].str[:-4]
trump_df['ZIP_CODE'] = trump_df['ZIP_CODE'].apply('{:0>5}'.format)

# Group TRANSACTION_AMT about ZIP_CODE and STATE and sort
# trump_df = trump_df.groupby(['ZIP_CODE', 'STATE'])['TRANSACTION_AMT'].median().to_frame().reset_index()

# Sort values by ZIP_CODE and STATE for viewing
trump_df = trump_df.sort_values(['ZIP_CODE', 'STATE'])

# Change dtype for manipulation
trump_df['ZIP_CODE'] = trump_df['ZIP_CODE'].apply(np.int64)

# Remove invalid zip codes
trump_df = trump_df[(trump_df['ZIP_CODE'] > 986) & (trump_df['ZIP_CODE'] < 99950)]

# Reset index
trump_df = trump_df.reset_index(drop=True)

trump_df.drop([199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209], axis=0, inplace=True)

# Reformat ZIP_CODE to five characters
trump_df['ZIP_CODE'] = trump_df['ZIP_CODE'].apply(lambda x: '{0:0>5}'.format(x))

# Reset index
trump_df = trump_df.reset_index(drop=True)

trump_df

,STATE,ZIP_CODE,TRANSACTION_AMT
0,MA,01001,45
1,MA,01001,475
2,MA,01001,25
3,MA,01001,100
4,MA,01001,2500
...,...,...,...
4905465,AK,99929,266
4905466,AK,99929,50
4905467,AK,99929,100
4905468,AK,99929,20


In [10]:
q = """
SELECT t.ZIP_CODE, t.STATE, t.TRANSACTION_AMT, z.STATE_ID, z.CD
FROM trump_df as t
LEFT JOIN zip_df as z
    ON t.ZIP_CODE = z.ZIP AND t.STATE = z.STATE
"""
trump_df = psql.sqldf(q)
# Group TRANSACTION_AMT by below values, sort, reset index
trump_df = trump_df.groupby(['STATE', 'STATE_ID', 'CD'])['TRANSACTION_AMT'].mean().to_frame().reset_index()
trump_df = trump_df.sort_values(['STATE_ID', 'CD'])
trump_df = trump_df.reset_index(drop=True)
trump_df.loc[trump_df['CD'] == '00', 'CD'] = '01'
trump_df

,STATE,STATE_ID,CD,TRANSACTION_AMT
0,AL,01,01,134.108567
1,AL,01,02,205.423998
2,AL,01,03,155.721515
3,AL,01,04,114.180377
4,AL,01,05,123.600812
...,...,...,...,...
431,WI,55,05,207.390905
432,WI,55,06,222.275017
433,WI,55,07,222.590395
434,WI,55,08,195.781006


In [11]:
# # Group TRANSACTION_AMT by below values, sort, reset index
# trump_df = trump_df.groupby(['STATE', 'STATE_ID', 'CD'])['TRANSACTION_AMT'].mean().to_frame().reset_index()
# trump_df = trump_df.sort_values(['STATE_ID', 'CD'])
# trump_df = trump_df.reset_index(drop=True)
# trump_df.loc[trump_df['CD'] == '00', 'CD'] = '01'
# trump_df

In [12]:
filename = '/Users/darionguan/Desktop/Presidential Prediction/acs_cd_final.csv'
acs_df = pd.read_csv(filename).iloc[: , 1:]
acs_df['FIPS'] = acs_df['FIPS'].apply('{:0>2}'.format)
acs_df['CD'] = acs_df['CD'].apply('{:0>2}'.format)

acs_df

,State,STATE_ABB,FIPS,CD,MedianHouseholdIncome,MedianNonfamilyIncome,Pct18to64InPoverty,PctOver65InPoverty,PctUnemployedCivilians,PctLessThan9thGrade,Pct9thto12thGrade,PctHSGrad,PctSomeCollege,PctAssociatesDeg,PctBachelorsDeg,PctGradDeg
0,Alabama,AL,01,01,50663,30805,0.144,0.102,0.057,0.029,0.089,0.336,0.207,0.089,0.161,0.089
1,Alabama,AL,01,02,50494,31799,0.156,0.088,0.045,0.041,0.093,0.319,0.218,0.094,0.148,0.087
2,Alabama,AL,01,03,51925,25093,0.154,0.102,0.059,0.047,0.102,0.309,0.205,0.097,0.136,0.104
3,Alabama,AL,01,04,47531,22794,0.147,0.112,0.043,0.052,0.119,0.342,0.220,0.092,0.115,0.061
4,Alabama,AL,01,05,59950,33709,0.128,0.079,0.043,0.034,0.072,0.272,0.200,0.087,0.209,0.126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,Wisconsin,WI,55,05,77386,44574,0.060,0.070,0.032,0.018,0.033,0.263,0.200,0.105,0.260,0.123
433,Wisconsin,WI,55,06,63251,40066,0.070,0.079,0.025,0.019,0.049,0.337,0.199,0.113,0.194,0.088
434,Wisconsin,WI,55,07,60706,33054,0.092,0.076,0.029,0.023,0.052,0.345,0.210,0.126,0.167,0.077
435,Wisconsin,WI,55,08,65346,38058,0.081,0.068,0.029,0.027,0.043,0.335,0.202,0.120,0.191,0.081


In [13]:
q = """
SELECT b.STATE,
        b.STATE_ID,
        b.CD,
        b.TRANSACTION_AMT,
        a.MedianHouseholdIncome,
        a.MedianNonfamilyIncome,
        a.Pct18to64InPoverty,
        a.PctOver65InPoverty,
        a.PctUnemployedCivilians,
        a.PctLessThan9thGrade,
        a.Pct9thto12thGrade,
        a.PctHSGrad,
        a.PctSomeCollege,
        a.PctAssociatesDeg,
        a.PctBachelorsDeg,
        a.PctGradDeg
FROM biden_df AS b
LEFT JOIN acs_df AS a
    ON b.STATE_ID = a.FIPS AND b.CD = a.CD
"""

biden_df = psql.sqldf(q)
biden_df.to_csv('biden_donations.csv')
biden_df

,STATE,STATE_ID,CD,TRANSACTION_AMT,MedianHouseholdIncome,MedianNonfamilyIncome,Pct18to64InPoverty,PctOver65InPoverty,PctUnemployedCivilians,PctLessThan9thGrade,Pct9thto12thGrade,PctHSGrad,PctSomeCollege,PctAssociatesDeg,PctBachelorsDeg,PctGradDeg
0,AL,01,01,153.441706,50663,30805,0.144,0.102,0.057,0.029,0.089,0.336,0.207,0.089,0.161,0.089
1,AL,01,02,110.557545,50494,31799,0.156,0.088,0.045,0.041,0.093,0.319,0.218,0.094,0.148,0.087
2,AL,01,03,68.553998,51925,25093,0.154,0.102,0.059,0.047,0.102,0.309,0.205,0.097,0.136,0.104
3,AL,01,04,116.225519,47531,22794,0.147,0.112,0.043,0.052,0.119,0.342,0.220,0.092,0.115,0.061
4,AL,01,05,100.808035,59950,33709,0.128,0.079,0.043,0.034,0.072,0.272,0.200,0.087,0.209,0.126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,WI,55,05,124.336661,77386,44574,0.060,0.070,0.032,0.018,0.033,0.263,0.200,0.105,0.260,0.123
432,WI,55,06,132.124739,63251,40066,0.070,0.079,0.025,0.019,0.049,0.337,0.199,0.113,0.194,0.088
433,WI,55,07,80.208220,60706,33054,0.092,0.076,0.029,0.023,0.052,0.345,0.210,0.126,0.167,0.077
434,WI,55,08,108.448556,65346,38058,0.081,0.068,0.029,0.027,0.043,0.335,0.202,0.120,0.191,0.081


In [14]:
q = """
SELECT t.STATE,
        t.STATE_ID,
        t.CD,
        t.TRANSACTION_AMT,
        a.MedianHouseholdIncome,
        a.MedianNonfamilyIncome,
        a.Pct18to64InPoverty,
        a.PctOver65InPoverty,
        a.PctUnemployedCivilians,
        a.PctLessThan9thGrade,
        a.Pct9thto12thGrade,
        a.PctHSGrad,
        a.PctSomeCollege,
        a.PctAssociatesDeg,
        a.PctBachelorsDeg,
        a.PctGradDeg
FROM trump_df AS t
LEFT JOIN acs_df AS a
    ON t.STATE_ID = a.FIPS AND t.CD = a.CD
"""

trump_df = psql.sqldf(q)
trump_df.to_csv('trump_donations.csv')
trump_df

,STATE,STATE_ID,CD,TRANSACTION_AMT,MedianHouseholdIncome,MedianNonfamilyIncome,Pct18to64InPoverty,PctOver65InPoverty,PctUnemployedCivilians,PctLessThan9thGrade,Pct9thto12thGrade,PctHSGrad,PctSomeCollege,PctAssociatesDeg,PctBachelorsDeg,PctGradDeg
0,AL,01,01,134.108567,50663,30805,0.144,0.102,0.057,0.029,0.089,0.336,0.207,0.089,0.161,0.089
1,AL,01,02,205.423998,50494,31799,0.156,0.088,0.045,0.041,0.093,0.319,0.218,0.094,0.148,0.087
2,AL,01,03,155.721515,51925,25093,0.154,0.102,0.059,0.047,0.102,0.309,0.205,0.097,0.136,0.104
3,AL,01,04,114.180377,47531,22794,0.147,0.112,0.043,0.052,0.119,0.342,0.220,0.092,0.115,0.061
4,AL,01,05,123.600812,59950,33709,0.128,0.079,0.043,0.034,0.072,0.272,0.200,0.087,0.209,0.126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,WI,55,05,207.390905,77386,44574,0.060,0.070,0.032,0.018,0.033,0.263,0.200,0.105,0.260,0.123
432,WI,55,06,222.275017,63251,40066,0.070,0.079,0.025,0.019,0.049,0.337,0.199,0.113,0.194,0.088
433,WI,55,07,222.590395,60706,33054,0.092,0.076,0.029,0.023,0.052,0.345,0.210,0.126,0.167,0.077
434,WI,55,08,195.781006,65346,38058,0.081,0.068,0.029,0.027,0.043,0.335,0.202,0.120,0.191,0.081
